## Alchemy Core

### 创建连接

#### 导入包

In [1]:
from sqlalchemy import create_engine,Integer,String,Numeric,DateTime,Boolean,Table,Column,MetaData
from sqlalchemy import ForeignKey
from datetime import datetime
from sqlalchemy import insert,update,delete,select

#### 创建元数据Metadata对象

In [2]:
metadata = MetaData()

#### 创建表

In [3]:

cookies = Table('cookies',metadata,
                    Column('cookie_id',Integer(),primary_key=True),
                    Column('cookie_name',String(50),index=True),
                    Column('cookie_recipe_url',String(255)),
                    Column('cookie_sku',String(55)),
                    Column('quantity',Integer()),
                    Column('unit_cost',Numeric(12,2))
                    )
users = Table('users',metadata,
                  Column('user_id',Integer(),primary_key=True),
                  Column('customer_number',Integer(),autoincrement=True),
                  Column('user_name',String(15),nullable=False,unique=True),
                  Column('email_address',String(255),nullable=False),
                  Column('password',String(25),nullable=False),
                  Column('created_on',DateTime(),default=datetime.now),
                  Column('updated_on',DateTime(),default=datetime.now,onupdate=datetime.now)
                  )
orders = Table('orders',metadata,
                   Column('order_id',Integer(),primary_key=True),
                   Column('user_id',ForeignKey('users.user_id')),
                    Column('shipped',Boolean(),default=False)
                          )

line_items = Table('line_items',metadata,
                       Column('line_item_id',Integer(),primary_key=True),
                       Column('order_id',ForeignKey('orders.order_id')),
                       Column('cookie_id',ForeignKey('cookies.cookie_id')),
                       Column('quantity',Integer()),
                       Column('extended_cost',Numeric(12,2)))

#### 持久化

In [4]:
engine = create_engine("sqlite:///cookies.db")
metadata.create_all(engine)

### 插入数据

In [ ]:
ins = insert(cookies).values(cookie_name = 'chocolate chip',
                                 cookie_recipe_url='http://some.awson.me/cookie/recipe.html',
                                 cookie_sku='CC01',
                                 quantity = '12',
                                 unit_cost = '0.50')

print(ins)
result = engine.execute(ins)
print("insert result %s" % result)

In [ ]:
inventory_list = [
   {
       'cookie_name':'peanut butter',
                                 'cookie_recipe_url':'http://some.awson.me/cookie/peanut.html',
                                 'cookie_sku':'PB01',
                                 'quantity' : "24",
                                 'unit_cost' : "0.25"
   } ,
     {
       'cookie_name' : 'oatmeal raisin',
                                 'cookie_recipe_url':'http://some.awson.me/cookie/raisin.html',
                                 'cookie_sku':'EB01',
                                 'quantity' : "100",
                                 'unit_cost' : "1.00"
   }
]
ins = cookies.insert()

result = engine.execute(ins,inventory_list)

### 查询

In [5]:
from sqlalchemy import select

s = select([cookies])
print(str(s))
rp = engine.execute(s)
results = rp.fetchall()
print(results)

SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_recipe_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
[(1, 'chocolate chip', 'http://some.awson.me/cookie/recipe.html', 'CC01', 512, Decimal('0.50')), (2, 'peanut butter', 'http://some.awson.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25')), (5, 'oatmeal raisin', 'http://some.awson.me/cookie/raisin.html', 'EB01', 100, Decimal('1.00'))]


/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/sql/sqltypes.py:661: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


In [ ]:
first_row = results[0]
first_row.cookie_name

#### resultProxy

In [ ]:
s = select([cookies])
rp = engine.execute(s)
for record in rp:
    print(record.cookie_name)

In [ ]:
s = select([cookies])
rp = engine.execute(s)

In [ ]:
rp.first()

In [ ]:
rp.fetchone()

#### 查询指定的列 加排序


In [ ]:
s = select([cookies.c.cookie_name,cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
rp = engine.execute(s)
for cookie in rp:
    print('{}-{}'.format(cookie.quantity,cookie.cookie_name))

#### 限制返回的条数

In [ ]:
from sqlalchemy import desc
s = select([cookies.c.cookie_name,cookies.c.quantity])
s = s.order_by(desc(cookies.c.quantity))
s = s.limit(2)
rp = engine.execute(s)
print([result.cookie_name for result in rp])

#### 内置sql函数和标签

In [ ]:
from sqlalchemy.sql import func
s = select([func.sum(cookies.c.quantity)])
rp = engine.execute(s)
print(rp.scalar())

In [ ]:
from sqlalchemy.sql import func
s = select([func.count(cookies.c.cookie_name)])
rp = engine.execute(s)
record = rp.first()
print(record.keys())
print(record.count_1)

In [ ]:
### 使用label重命名
s = select([func.count(cookies.c.cookie_name).label("inventory_count")])
rp = engine.execute(s)
record = rp.first()
print(record.keys())
print(record.inventory_count)

#### 过滤

In [ ]:
s = select([cookies]).where(cookies.c.cookie_name == 'chocolate chip')
rp = engine.execute(s)
record = rp.first()
print(record.items())

In [ ]:
s = select([cookies]).where(cookies.c.cookie_name.like("%chip%"))
rp = engine.execute(s)
for record in rp.fetchall():
    print(record.cookie_name)

#### 运算符

In [ ]:
s = select([cookies.c.cookie_name,'SKU'+cookies.c.cookie_sku])
for row in engine.execute(s):
    print(row)

In [ ]:
from sqlalchemy import cast
s = select([cookies.c.cookie_name,cast((cookies.c.quantity*cookies.c.unit_cost),Numeric(12,2)).label('inv_cost')])

for row in engine.execute(s):
    print("{}-{}".format(row.cookie_name,row.inv_cost))

In [ ]:
#### 布尔运算符
from sqlalchemy import and_,or_,not_
s = select([cookies]).where(
and_(cookies.c.quantity > 23,cookies.c.unit_cost<0.4))
for row in engine.execute(s):
    print(row.cookie_name)

### 更新 

In [ ]:
u= update(cookies).where(cookies.c.cookie_name == 'chocolate chip')
u = u.values(quantity = (cookies.c.quantity + 100))
result = engine.execute(u)
print(result.rowcount)

s = select([cookies]).where(cookies.c.cookie_name == 'chocolate chip')
result = engine.execute(s).first()
print(result)
for key in result.keys():
    print('{:>20}:{}'.format(key,result[key]))

### 删除

In [ ]:
from sqlalchemy import delete
u = delete(cookies).where(cookies.c.cookie_id == 4)
reuslt = engine.execute(u)
print(reuslt.rowcount)

In [ ]:
customer_list = [
    {
        'user_name':'cookiemon',
        'email_address':'mo@cookie.com',
        'password':'123456',
    },
      {
        'user_name':'cakeeater',
        'email_address':'cakeeater@cookie.com',
        'password':'123456',
    },
      {
        'user_name':'pieguy',
        'email_address':'pieguy@cookie.com',
        'password':'123456',
    }
]

ins = users.insert()
result = engine.execute(ins,customer_list)


In [6]:
s = select([users])
engine.execute(s).fetchall()

[(1, None, 'cookiemon', 'mo@cookie.com', '123456', datetime.datetime(2021, 7, 22, 21, 39, 24, 273099), datetime.datetime(2021, 7, 22, 21, 39, 24, 273108)),
 (2, None, 'cakeeater', 'cakeeater@cookie.com', '123456', datetime.datetime(2021, 7, 22, 21, 39, 24, 273110), datetime.datetime(2021, 7, 22, 21, 39, 24, 273112)),
 (3, None, 'pieguy', 'pieguy@cookie.com', '123456', datetime.datetime(2021, 7, 22, 21, 39, 24, 273114), datetime.datetime(2021, 7, 22, 21, 39, 24, 273115))]

In [9]:
ins = insert(orders).values(user_id = 2,order_id = 2)
print(str(ins))
with engine.connect() as connection:
    result = connection.execute(ins)
    print(result)

INSERT INTO orders (order_id, user_id, shipped) VALUES (:order_id, :user_id, :shipped)


In [13]:
order_items = [
    {
        'order_id':2,
        'cookie_id':1,
        'quantity':24,
        'extended_cost':12.00,
        
    },
    
      {
        'order_id':2,
        'cookie_id':4,
        'quantity':6,
        'extended_cost':6.00,
        
    }]
ins = insert(line_items)
connection = engine.connect()
connection.execute(ins,order_items)
    

#### join连接

In [26]:
connection = engine.connect()
columns = [orders.c.order_id,users.c.user_name,users.c.email_address,cookies.c.cookie_name,line_items.c.quantity,line_items.c.extended_cost]
cookiemon_orders = select(columns)咯；p
cookiemon_orders = cookiemon_orders.select_from(orders.join(users).join(line_items).join(cookies)).where(users.c.user_name=='cookiemon')
print(str(cookiemon_orders))
rp = connection.execute(cookiemon_orders)
result = rp.fetchall()
for row in result:
    print(row)
connection.close()

SELECT orders.order_id, users.user_name, users.email_address, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.user_name = :user_name_1
(1, 'cookiemon', 'mo@cookie.com', 'chocolate chip', 2, Decimal('1.00'))
